In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import random
import gymnasium as gym
from collections import deque
from collections import defaultdict
import matplotlib.pyplot as plt
from test_script import QNetwork
from test_script import bar_plot, test_pole_length, test_script
import os
import json


# Main Classes used in different strategies

In [2]:
#replay buffer
class ReplayBuffer:
    """
    Replay Buffer to store experience tuples for deep q learning.
    The replay buffer stores experiences from many episodes and randomly samples them during training.
    """
    def __init__(self, capacity):
        self.buffer = deque(maxlen=capacity)
        self.sampled_pole_lengths = []
    
    def push(self, state, action, reward, next_state, done, pole_length):
        self.buffer.append((state, action, reward, next_state, done, pole_length))
    
    def sample(self, batch_size):
        return random.sample(self.buffer, batch_size)
    
    def __len__(self):
        return len(self.buffer)

In [3]:


# --- MODIFIED REPLAY BUFFER FOR PER ---
class PrioritizedReplayBuffer:
    """
    Prioritized Replay Buffer using a simple list for storage and management.
    NOTE: A proper, efficient PER implementation uses a SumTree/SegmentTree.
    This simplified version is for conceptual demonstration but is O(N) for sampling.
    """
    def __init__(self, capacity, alpha=0.6, epsilon=1e-5):
        self.capacity = capacity
        # Stored transitions
        self.buffer = []
        # Stored priorities (parallel list)
        self.priorities = []
        self.alpha = alpha  # PER exponent
        self.epsilon = epsilon # Small constant to ensure non-zero probability
        self.position = 0
        self.sampled_pole_lengths = []

    def push(self, state, action, reward, next_state, done, pole_length):
        max_priority = 1.0 if not self.priorities else max(self.priorities)
        sample = (state, action, reward, next_state, done, pole_length)
        
        # If buffer is not full, append; otherwise, overwrite old sample
        if len(self.buffer) < self.capacity:
            self.buffer.append(sample)
            self.priorities.append(max_priority) # NEW: Assign max priority to new sample
        else:
            self.buffer[self.position] = sample
            self.priorities[self.position] = max_priority # NEW: Assign max priority to overwritten sample
        
        self.position = (self.position + 1) % self.capacity

    def sample(self, batch_size, beta=0.4):
        # Calculate P(i) based on priorities
        # p_i = (priority + epsilon)^alpha
        # P(i) = p_i / sum(p_k)
        # here we are calculating the priorrities for the weights which will be used in training and sampling

        scaled_priorities = np.array(self.priorities)**self.alpha
        sum_priorities = scaled_priorities.sum()
        prob_i = scaled_priorities / sum_priorities

        indices = np.random.choice(len(self.buffer), batch_size, p=prob_i)
        
        batch = [self.buffer[i] for i in indices]
        

        # here we calculate the sampling weights based on importance
        N = len(self.buffer)
        weights = (1.0 / N / prob_i[indices])**beta
        
        # Normalize weights by the maximum weight to stabilize training
        max_weight = weights.max()
        weights = weights / max_weight

        return batch, indices, weights # NEW: Return indices and weights

    def update_priorities(self, indices, errors): # NEW: Method to update priorities
        for idx, error in zip(indices, errors):
            # Prioritized Experience Replay uses the absolute TD error for priority
            self.priorities[idx] = max(error, self.epsilon)

    def __len__(self):
        return len(self.buffer)

In [4]:
class AdaptiveCurriculumLearning:
    """ 
    Adaptive curriculum learning offers an object to store the accumulated rewards, 
    performances, difficulty socores, and probabilities for each pole length. 
    We check the last n (LOOK_BACK_WINDOW) rewards a pole has obtained and assign probabilities 
    to sample each length for an episode. The goal is to attack the policy on its weak spots, 
    we prioritize training on weak performing lengths. This should shift performance metrics and keeps 
    our attacks adaptive as we target weakest. 
    param: all_pole_lengths : numpy.ndarray - stores all the pole lengths
    attr: p_adaptive : float - probability to select pole length from sample distribution, after uniform_episode_training_cap
    attr: lb_window : int - look back window used to compute performance metric, number of recent pole length performances    
    """
    LOOK_BACK_WINDOW = 20 # we only consider the last 20 rewards a pole has gotten (captures more sensitive information)
    P_ADAPTIVE = 0.9 # probability of using adaptive probability distribution or uniform random
    def __init__(self, all_pole_lengths):
        self.all_pole_lengths = all_pole_lengths
        self.rewards = defaultdict(list) # storing pole length episode reward for approach 2 (see 2.2)
        self.performances = {} # keep track of performance metric for each pole length 
        self.difficulty_scores = {} # difficulty_scores
        self.distribution = {} # probability distribution for pole lengths to sample from
        
        # initialize difficulties and probabilities (initially prob is uniform)
        initial_prob = 1.0 / len(all_pole_lengths)
        self.i_p = initial_prob
        for length in all_pole_lengths:
            self.difficulty_scores[length] = 1.0
            self.distribution[length] = initial_prob

    def update_rewards(self, pole_length, reward):
        self.rewards[pole_length].append(reward)

    def update_performances(self, pole_length):
        """
        Here we update the performance of a pole, 
        """
        reward_list = self.rewards[pole_length]

        # if a pole has not been played yet, it will be assigned a performance metric of 0
        if not reward_list:
            metric = 0
        else:
            # here we use a LOOK BACK WINDOW so that we dont use over-stabilized episode rewards
            metric = np.mean(reward_list[-self.LOOK_BACK_WINDOW:])
        self.performances[pole_length] = metric

    def update_difficulties(self):
        """
        Difficulties are inversely proportional to the performance metrics.
        The worse a pole length has performed, the higher the diff score (diff=1 being the worst performing, diff=0 being best).
        These require global updates as the update is relative to the totality of pole lengths.
        Also normalizing + scaling the values down. Metrics can offer quite larger values otherwise. 
        """
        M_max = self.find_max()
        M_min = self.find_min()
        diff_M = M_max - M_min

        # update all difficulty scores with new information
        for pole_length, metric in self.performances.items():
            if diff_M == 0:
                difficulty = 1
            else:
                difficulty = 1 - ((metric-M_min) / diff_M) # if best perform, diff is 0 >>> probability assignment will be 0
            self.difficulty_scores[pole_length] = difficulty
    
    def update_distribution(self):
        """ 
        Distribution is also global, here we require an update proportional to the totality. 
        Normalizing the probabilities (between 0 and 1)
        """
        if not self.difficulty_scores:
            return
        total_difficulty = sum(self.difficulty_scores.values())

        if total_difficulty > 0:
            for pole_length, difficulty in self.difficulty_scores.items():
                self.distribution[pole_length] = difficulty / total_difficulty
        else:
            for pole_length in self.all_pole_lengths:
                self.distribution[pole_length] = self.i_p
    
    def sample_length(self):
        """
        Here select the pole length using either uniform or categorical prob distribution. 
        """
        if random.random() < self.P_ADAPTIVE:
            pole_lengths = list(self.distribution.keys())
            probs = list(self.distribution.values())

            # if our prob dist is empty, we fallback to uniform selection
            if not pole_lengths or sum(probs) == 0:
                return np.random.choice(self.all_pole_lengths)
            
            # selection based on categorical sampling (discrete prob distribution)
            return np.random.choice(a=pole_lengths, p=probs, size=1)[0]
        else:
            return np.random.choice(self.all_pole_lengths)

    def find_max(self):
        """
        Find max performing pole length
        """
        if not self.performances:
            return 0
        return max(self.performances.values())

    def find_min(self):
        """
        Find min performing pole length
        """
        if not self.performances:
            return 0
        return min(self.performances.values())
    
    def calculate_pole_stats(self):
        """
        Nice display function for avg reward of each pole. Not functionally important at all. 
        """
        avg_pole_stats = {}
        
        for pole_length, rewards_list in self.rewards.items():
            # Only process if the list is not empty
            if rewards_list:
                avg_reward = np.mean(rewards_list)
                count = len(rewards_list)
            else:
                avg_reward = 0.0
                count = 0
            avg_pole_stats[pole_length] = {
                "average_reward": avg_reward,
                "episode_count": count
            }
        for p, avg in avg_pole_stats.items():
            print(f"Pole length {p} has avg reward {avg}")
        return avg_pole_stats


In [5]:
def select_action(state, policy_net, epsilon, action_dim):
    """
    Select action using epsilon-greedy policy - did it with epsilon-greedy because of Assignent 1
    """
    if random.random() < epsilon:
        return random.randrange(action_dim)
    else:
        with torch.no_grad():
            state_tensor = torch.FloatTensor(state).unsqueeze(0)
            q_values = policy_net(state_tensor)
            return q_values.argmax().item()

# Baseline model (A) 

In [6]:
def deep_q_learning(epsilon, gamma, alpha, q_network, n_episodes, 
                    pole_lengths=None, env_name='CartPole-v1',
                    batch_size=64, buffer_capacity=50000, 
                    update_target_every=1000, update_weights_every=4, 
                    epsilon_decay=1, epsilon_min=0.01,
                    per_alpha=0.6, per_beta_init=0.4
                    ):
    """
    Deep q learning agent for CartPole-v1 environment with varying pole lengths.
    
    param: epsilon : float - initial exploration rate
    param: gamma : float - discount factor
    param: alpha : float - learning rate
    param: q_network : QNetwork or None - pre-initialized network or None to create new one
    param: n_episodes : int - number of training episodes
    param: pole_lengths : array-like or None - array of pole lengths to train on (default: linspace(0.4, 1.8, 30))
    param: env_name : str - gym environment name
    param: batch_size : int - batch size for training
    param: buffer_capacity : int - replay buffer capacity
    param: update_target_every : int - how often to update target network
    param: epsilon_decay : float - epsilon decay rate per episode
    param: epsilon_min : float - minimum epsilon value

    return: tuple : (policy_net, target_net, episode_returns)
        - trained networks and list of episode rewards
    """

    # initialization of environment
    env = gym.make(env_name)
    state_dim = env.observation_space.shape[0]
    action_dim = env.action_space.n

    # initialization of networks if not provided
    if q_network is None:
        policy_net = QNetwork(state_dim, action_dim)
        target_net = QNetwork(state_dim, action_dim)
        target_net.load_state_dict(policy_net.state_dict())
        target_net.eval()
    else:
        policy_net = q_network
        target_net = QNetwork(state_dim, action_dim)
        target_net.load_state_dict(policy_net.state_dict())
        target_net.eval()

    # initialization of optimizer
    optimizer = optim.Adam(policy_net.parameters(), lr=alpha)

    # initialization of replay buffer
    replay_buffer = ReplayBuffer(buffer_capacity)
    
    # pole lengths for training
    if pole_lengths is None:
        pole_lengths = np.linspace(0.4, 1.8, 30)

    # storing episode returns for plotting
    episode_returns = []

    # storing p lengths
    visited_pole_lengths = []
    
    # copy of current epsilon value for decay
    epsi = epsilon

    # set counts to keep track for updating the weights/ and target net
    count = 0 
    
    # training loop
    for episode in range(n_episodes):
        # randomly select pole length for this episode (we need to figure an experimental setup)
        pole_length = np.random.choice(pole_lengths)
        env.unwrapped.length = pole_length
        
        # reset environment
        state = env.reset()[0]
        episode_reward = 0.0
        
        # episode loop (1 episode = 1 pole length)
        done = False

        while not done:
            
            # increment count
            count += 1

            # select action
            action = select_action(state, policy_net, epsi, action_dim)
            
            # take step
            next_state, reward, terminated, truncated, _ = env.step(action)
            done = terminated or truncated

            # epsilon decay
            if epsi > epsilon_min:
                epsi = max(epsilon_min, epsi * epsilon_decay)

            # store transition in replay buffer
            replay_buffer.push(state, action, reward, next_state, float(done), pole_length)

            # deep q learning update (using mini-batch from replay buffer)
            if len(replay_buffer) >= batch_size:
                # sample batch from replay buffer
                if count % update_weights_every == 0:
                    batch = replay_buffer.sample(batch_size) 
                    states, actions, rewards, next_states, dones, labels_pole_length = zip(*batch)

                    # store the pole lengths in our self.sampled p lengths for later use
                    replay_buffer.sampled_pole_lengths.extend(labels_pole_length)

                    # convert to tensors 
                    states_t = torch.FloatTensor(states)
                    actions_t = torch.LongTensor(actions).unsqueeze(1)
                    rewards_t = torch.FloatTensor(rewards).unsqueeze(1)
                    next_states_t = torch.FloatTensor(next_states)
                    dones_t = torch.FloatTensor(dones).unsqueeze(1)

                    #get current q values
                    current_q = policy_net(states_t).gather(1, actions_t)
                    
                    # target values
                    with torch.no_grad():
                        next_max = target_net(next_states_t).max(1)[0].unsqueeze(1)
                        td_target = rewards_t + gamma * next_max * (1 - dones_t)
                    
                    # loss calc
                    loss = nn.MSELoss()(current_q, td_target)
                    
                    # backprop and optimize + gradient clipping
                    optimizer.zero_grad()
                    loss.backward()
                    torch.nn.utils.clip_grad_norm_(policy_net.parameters(), 1.0)
                    optimizer.step()
            
            # update target network periodically
            if count % update_target_every == 0:
                target_net.load_state_dict(policy_net.state_dict())
            episode_reward += reward
            state = next_state

        #store episode reward
        episode_returns.append(episode_reward)

        # store pole legnth
        visited_pole_lengths.append(pole_length)
        
        #only for seeing the progress
        if episode % 100 == 0:
            avg_reward = np.mean(episode_returns[-100:]) if len(episode_returns) >= 100 else np.mean(episode_returns)
            print(f"Episode {episode}/{n_episodes} | "
                  f"Avg Reward: {avg_reward:.1f} | "
                  f"Epsilon: {epsi:.3f}")
    
    env.close()

    sampled_pole_lengths = replay_buffer.sampled_pole_lengths
    
    return (policy_net, target_net, episode_returns, sampled_pole_lengths, visited_pole_lengths)

In [17]:
#Test the deep q learning function
# --- IGNORE ---

seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)

# --- training test ---
policy_net, target_net, returns, sampled_pole_lengths, visited_pole_lengths = deep_q_learning(
    epsilon= 0.1, 
    gamma=0.99, 
    alpha=1e-3, 
    q_network=None, 
    n_episodes=2000, 
    pole_lengths=None, 
    env_name='CartPole-v1',
    batch_size=64, 
    buffer_capacity=50000, 
    update_target_every=1000, 
    update_weights_every=4, 
    epsilon_decay=1, 
    epsilon_min=0.05,
    per_alpha=0.6, 
    per_beta_init=0.4
)

print("----finished training----")
print("Last 3 episode returns:", returns[-3:] if len(returns) >= 3 else returns)

Episode 0/2000 | Avg Reward: 18.0 | Epsilon: 0.100
Episode 100/2000 | Avg Reward: 19.5 | Epsilon: 0.100
Episode 200/2000 | Avg Reward: 14.1 | Epsilon: 0.100
Episode 300/2000 | Avg Reward: 17.6 | Epsilon: 0.100
Episode 400/2000 | Avg Reward: 183.7 | Epsilon: 0.100


KeyboardInterrupt: 

`quick check that pole lengths are being returned well`

`saving the training data and model`


In [ ]:
SAVE_FOLDER = "trained_models_rb_strategy"
MODEL_NAME = "baseline.pth" # .pth or .pt are common extensions
SAVE_PATH = os.path.join(SAVE_FOLDER, MODEL_NAME)
if not os.path.exists(SAVE_FOLDER):
    os.makedirs(SAVE_FOLDER)
    print(f"Created directory: {SAVE_FOLDER}")

torch.save(policy_net.state_dict(), SAVE_PATH)

_SAVE_FOLDER = "training_data_version_2"
_DATA_NAME = "baseline_rewards.json" 
__DATA_NAME = "baseline_sampled_pole_lengths.json"# .pth or .pt are common extensions
os.makedirs(_SAVE_FOLDER, exist_ok=True)
full_path = os.path.join(_SAVE_FOLDER, _DATA_NAME)
_full_path = os.path.join(_SAVE_FOLDER, __DATA_NAME)
try:
    with open(full_path, 'w') as file:
        json.dump(returns, file, indent=4) # indent=4 makes the file nicely formatted
    print(f"List successfully saved to {full_path}")
except Exception as e:
    print(f"An error occurred: {e}")

try:
    with open(_full_path, 'w') as file:
        json.dump(sampled_pole_lengths, file, indent=4) # indent=4 makes the file nicely formatted
    print(f"List successfully saved to {_full_path}")
except Exception as e:
    print(f"An error occurred: {e}")

_visited_pole_lengths = "baseline_visited.json"
v_path = os.path.join(_SAVE_FOLDER, _visited_pole_lengths)

try:
    with open(v_path, 'w') as file:
        json.dump(visited_pole_lengths, file, indent=4) # indent=4 makes the file nicely formatted
    print(f"List successfully saved to {v_path}")
except Exception as e:
    print(f"An error occurred: {e}")

List successfully saved to trained_nn_version_2\baseline_5k_rewards.json
List successfully saved to trained_nn_version_2\baseline_5k_sampled_pole_lengths.json


# Prioritized Experience Replay strategy

In [10]:


def deep_q_learning(epsilon, gamma, alpha, q_network, n_episodes, 
                    pole_lengths=None, env_name='CartPole-v1',
                    batch_size=64, buffer_capacity=50000, 
                    update_target_every=1000, update_weights_every=4, 
                    epsilon_decay=0.995, epsilon_min=0.01,
                    per_alpha=0.6, per_beta_init=0.4
                    ):
    """
    Deep q learning agent for CartPole-v1 environment with varying pole lengths.
    
    param: epsilon : float - initial exploration rate
    param: gamma : float - discount factor
    param: alpha : float - learning rate
    param: q_network : QNetwork or None - pre-initialized network or None to create new one
    param: n_episodes : int - number of training episodes
    param: pole_lengths : array-like or None - array of pole lengths to train on (default: linspace(0.4, 1.8, 30))
    param: env_name : str - gym environment name
    param: batch_size : int - batch size for training
    param: buffer_capacity : int - replay buffer capacity
    param: update_target_every : int - how often to update target network
    param: epsilon_decay : float - epsilon decay rate per episode
    param: epsilon_min : float - minimum epsilon value

    return: tuple : (policy_net, target_net, episode_returns)
        - trained networks and list of episode rewards
    """

    # initialization of environment
    env = gym.make(env_name)
    state_dim = env.observation_space.shape[0]
    action_dim = env.action_space.n

    # initialization of networks if not provided
    if q_network is None:
        policy_net = QNetwork(state_dim, action_dim)
        target_net = QNetwork(state_dim, action_dim)
        target_net.load_state_dict(policy_net.state_dict())
        target_net.eval()
    else:
        policy_net = q_network
        target_net = QNetwork(state_dim, action_dim)
        target_net.load_state_dict(policy_net.state_dict())
        target_net.eval()

    # initialization of optimizer
    optimizer = optim.Adam(policy_net.parameters(), lr=alpha)

    # initialization of replay buffer
    replay_buffer = PrioritizedReplayBuffer(buffer_capacity, per_alpha)
    
    # pole lengths for training
    if pole_lengths is None:
        pole_lengths = np.linspace(0.4, 1.8, 30)

    # storing episode returns for plotting
    episode_returns = []

    # storing pole lengths 
    visited_pole_lengths = []
    
    # copy of current epsilon value for decay
    epsi = epsilon

    # set counts to keep track for updating the weights/ and target net
    count = 0 

    total_steps = n_episodes * 500 # Estimate max steps for annealing (CartPole max steps is 500)
    beta = per_beta_init
    beta_increment = (1.0 - per_beta_init) / total_steps # Anneal beta to 1.0 over training

    
    # training loop
    for episode in range(n_episodes):
        # randomly select pole length for this episode (we need to figure an experimental setup)
        pole_length = np.random.choice(pole_lengths)
        env.unwrapped.length = pole_length
        
        # reset environment
        state = env.reset()[0]
        episode_reward = 0.0
        
        # episode loop (1 episode = 1 pole length)
        done = False

        while not done:
            
            # increment count
            count += 1

            # Anneal Beta for Importance Sampling (IS) weights
            beta = min(1.0, beta + beta_increment) 

            # select action
            action = select_action(state, policy_net, epsi, action_dim)
            
            # take step
            next_state, reward, terminated, truncated, _ = env.step(action)
            done = terminated or truncated

            # epsilon decay
            if epsi > epsilon_min:
                epsi = max(epsilon_min, epsi * epsilon_decay)

            # store transition in replay buffer
            replay_buffer.push(state, action, reward, next_state, float(done), pole_length)

            # deep q learning update (using mini-batch from replay buffer)
            if len(replay_buffer) >= batch_size:
                # sample batch from replay buffer
                if count % update_weights_every == 0:
                    batch, indices, weights = replay_buffer.sample(batch_size, beta) 
                    states, actions, rewards, next_states, dones, labels_pole_length = zip(*batch)

                    # store the pole lengths in our self.sampled p lengths for later use
                    replay_buffer.sampled_pole_lengths.extend(labels_pole_length)

                    # convert to tensors 
                    states_t = torch.FloatTensor(states)
                    actions_t = torch.LongTensor(actions).unsqueeze(1)
                    rewards_t = torch.FloatTensor(rewards).unsqueeze(1)
                    next_states_t = torch.FloatTensor(next_states)
                    dones_t = torch.FloatTensor(dones).unsqueeze(1)
                    weights_t = torch.FloatTensor(weights).unsqueeze(1) 

                    #get current q values
                    current_q = policy_net(states_t).gather(1, actions_t)
                    
                    # target values
                    with torch.no_grad():
                        next_max = target_net(next_states_t).max(1)[0].unsqueeze(1)
                        td_target = rewards_t + gamma * next_max * (1 - dones_t)
                        td_error = torch.abs(td_target - current_q).squeeze().detach().cpu().numpy()

                    
                    # loss calc
                    loss_unweighted = torch.nn.MSELoss(reduction='none')(current_q, td_target)
                    loss = (loss_unweighted * weights_t).mean()

                    # update priorities in the replay buffer
                    replay_buffer.update_priorities(indices, td_error) 
                    
                    # backprop and optimize + gradient clipping
                    optimizer.zero_grad()
                    loss.backward()
                    torch.nn.utils.clip_grad_norm_(policy_net.parameters(), 1.0)
                    optimizer.step()
            
            # update target network periodically
            if count % update_target_every == 0:
                target_net.load_state_dict(policy_net.state_dict())
            episode_reward += reward
            state = next_state

        #store episode reward
        episode_returns.append(episode_reward)
        
        # store pole legnth
        visited_pole_lengths.append(pole_length)

        #only for seeing the progress
        if episode % 100 == 0:
            avg_reward = np.mean(episode_returns[-100:]) if len(episode_returns) >= 100 else np.mean(episode_returns)
            print(f"Episode {episode}/{n_episodes} | "
                  f"Avg Reward: {avg_reward:.1f} | "
                  f"Epsilon: {epsi:.3f}")
    
    env.close()

    sampled_pole_lengths = replay_buffer.sampled_pole_lengths
    
    return (policy_net, target_net, episode_returns, sampled_pole_lengths, visited_pole_lengths)

In [15]:
#Test the deep q learning function
# --- IGNORE ---
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)

# --- training test ---
policy_net, target_net, returns, sampled_pole_lengths, visited_pole_lengths = deep_q_learning(
    epsilon= 0.1, 
    gamma=0.99, 
    alpha=1e-3, 
    q_network=None, 
    n_episodes=2000, 
    pole_lengths=None, 
    env_name='CartPole-v1',
    batch_size=64, 
    buffer_capacity=50000, 
    update_target_every=1000, 
    update_weights_every=4, 
    epsilon_decay=1, 
    epsilon_min=0.05,
    per_alpha=0.6, 
    per_beta_init=0.4
)

print("----finished training----")
print("Last 3 episode returns:", returns[-3:] if len(returns) >= 3 else returns)

Episode 0/2000 | Avg Reward: 18.0 | Epsilon: 0.100
Episode 100/2000 | Avg Reward: 19.0 | Epsilon: 0.100
Episode 200/2000 | Avg Reward: 18.3 | Epsilon: 0.100
Episode 300/2000 | Avg Reward: 82.2 | Epsilon: 0.100
Episode 400/2000 | Avg Reward: 265.8 | Epsilon: 0.100
Episode 500/2000 | Avg Reward: 207.1 | Epsilon: 0.100
Episode 600/2000 | Avg Reward: 129.6 | Epsilon: 0.100
Episode 700/2000 | Avg Reward: 137.8 | Epsilon: 0.100
Episode 800/2000 | Avg Reward: 191.0 | Epsilon: 0.100
Episode 900/2000 | Avg Reward: 347.7 | Epsilon: 0.100
Episode 1000/2000 | Avg Reward: 192.0 | Epsilon: 0.100
Episode 1100/2000 | Avg Reward: 245.8 | Epsilon: 0.100
Episode 1200/2000 | Avg Reward: 359.9 | Epsilon: 0.100
Episode 1300/2000 | Avg Reward: 417.4 | Epsilon: 0.100
Episode 1400/2000 | Avg Reward: 297.1 | Epsilon: 0.100
Episode 1500/2000 | Avg Reward: 230.4 | Epsilon: 0.100
Episode 1600/2000 | Avg Reward: 206.0 | Epsilon: 0.100
Episode 1700/2000 | Avg Reward: 388.7 | Epsilon: 0.100
Episode 1800/2000 | Avg Re

`saving training data and model`

In [ ]:
SAVE_FOLDER = "trained_nn_version_2"
MODEL_NAME = "per_strategy.pth" # .pth or .pt are common extensions
SAVE_PATH = os.path.join(SAVE_FOLDER, MODEL_NAME)
if not os.path.exists(SAVE_FOLDER):
    os.makedirs(SAVE_FOLDER)
    print(f"Created directory: {SAVE_FOLDER}")

torch.save(policy_net.state_dict(), SAVE_PATH)

_SAVE_FOLDER = "training_data_version_2"
_DATA_NAME = "per_strategy_rewards.json" 
__DATA_NAME = "per_strategy_rewards_sampled_pole_lengths.json"# .pth or .pt are common extensions
os.makedirs(_SAVE_FOLDER, exist_ok=True)
full_path = os.path.join(_SAVE_FOLDER, _DATA_NAME)
_full_path = os.path.join(_SAVE_FOLDER, __DATA_NAME)
try:
    with open(full_path, 'w') as file:
        json.dump(returns, file, indent=4) # indent=4 makes the file nicely formatted
    print(f"List successfully saved to {full_path}")
except Exception as e:
    print(f"An error occurred: {e}")

try:
    with open(_full_path, 'w') as file:
        json.dump(sampled_pole_lengths, file, indent=4) # indent=4 makes the file nicely formatted
    print(f"List successfully saved to {_full_path}")
except Exception as e:
    print(f"An error occurred: {e}")

_visited_pole_lengths = "per_strategy_visited.json"
v_path = os.path.join(_SAVE_FOLDER, _visited_pole_lengths)

try:
    with open(v_path, 'w') as file:
        json.dump(visited_pole_lengths, file, indent=4) # indent=4 makes the file nicely formatted
    print(f"List successfully saved to {v_path}")
except Exception as e:
    print(f"An error occurred: {e}")

List successfully saved to training_data_version_2\per_strategy_rewards.json
List successfully saved to training_data_version_2\per_strategy_rewards_sampled_pole_lengths.json
List successfully saved to training_data_version_2\per_visited.json


# Adaptive Curriculum Learning strategy

In [ ]:
def deep_q_learning(epsilon, gamma, alpha, q_network, n_episodes,
                    uniform_episode_training_cap,
                    pole_lengths=None, env_name='CartPole-v1',
                    batch_size=64, buffer_capacity=50000, 
                    update_target_every=1000, update_weights_every=4,
                    epsilon_decay=1, 
                    epsilon_min=0.05):
    """
    Deep q learning agent for CartPole-v1 environment with varying pole lengths.
    
    param: epsilon : float - initial exploration rate
    param: gamma : float - discount factor
    param: alpha : float - learning rate
    param: q_network : QNetwork or None - pre-initialized network or None to create new one
    param: n_episodes : int - number of training episodes
    param: uniform_episode_training_cap : Union[int, None] - number episodes trained with uniform length selection
    if None, no adaptive curriculum learning enabled
    param: pole_lengths : array-like or None - array of pole lengths to train on (default: linspace(0.4, 1.8, 30))
    param: env_name : str - gym environment name
    param: batch_size : int - batch size for training
    param: buffer_capacity : int - replay buffer capacity
    param: update_target_every : int - how often to update target network
    param: epsilon_decay : float - epsilon decay rate per episode
    param: epsilon_min : float - minimum epsilon value

    return: tuple : (policy_net, target_net, episode_returns)
        - trained networks and list of episode rewards
    """
    
    # initialization of environment
    env = gym.make(env_name)
    state_dim = env.observation_space.shape[0]
    action_dim = env.action_space.n

    # initialization of networks if not provided
    if q_network is None:
        policy_net = QNetwork(state_dim, action_dim)
        target_net = QNetwork(state_dim, action_dim)
        target_net.load_state_dict(policy_net.state_dict())
        target_net.eval()
    else:
        policy_net = q_network
        target_net = QNetwork(state_dim, action_dim)
        target_net.load_state_dict(policy_net.state_dict())
        target_net.eval()

    # initialization of optimizer
    optimizer = optim.Adam(policy_net.parameters(), lr=alpha)

    # initialization of replay buffer
    replay_buffer = ReplayBuffer(buffer_capacity)
    
    # pole lengths for training
    if pole_lengths is None:
        pole_lengths = np.linspace(0.4, 1.8, 30)

    # storing episode returns for plotting
    episode_returns = []

    # storing visited pole lengths 
    visited_pole_lengths = []

    # initialize the acl class for adaptive hyperparametre sampling
    acl = AdaptiveCurriculumLearning(pole_lengths)

    # copy of current epsilon value for decay
    epsi = epsilon

    # set counts to keep track for updating the weights/ and target net
    count = 0
    
    # training loop
    for episode in range(n_episodes):
        # if current episode is below the uniform_episode_training_cap we select from a uniform distr
        if episode <= uniform_episode_training_cap: 
            pole_length = np.random.choice(pole_lengths)
        # else use adaptive probability distrubition 
        else: 
            pole_length = acl.sample_length()

        env.unwrapped.length = pole_length
        
        # reset environment
        state = env.reset()[0]
        episode_reward = 0.0
        
        # episode loop (1 episode = 1 pole length)
        done = False
        
        while not done:

            # increment count
            count += 1

            # select action
            action = select_action(state, policy_net, epsi, action_dim)
            
            # take step
            next_state, reward, terminated, truncated, _ = env.step(action)
            done = terminated or truncated

            # store transition in replay buffer
            replay_buffer.push(state, action, reward, next_state, float(done), pole_length)

            # epsilon decay
            if epsi > epsilon_min:
                epsi = max(epsilon_min, epsi * epsilon_decay)

            # deep q learning update (using mini-batch from replay buffer)
            if len(replay_buffer) >= batch_size:
                # sample batch from replay buffer
                if count % update_weights_every == 0:
                    batch = replay_buffer.sample(batch_size)
                    states, actions, rewards, next_states, dones, labels_pole_length = zip(*batch)

                    # add pole length to buffer for later use
                    replay_buffer.sampled_pole_lengths.extend(labels_pole_length)
                    
                    # convert to tensors 
                    states_t = torch.FloatTensor(states)
                    actions_t = torch.LongTensor(actions).unsqueeze(1)
                    rewards_t = torch.FloatTensor(rewards).unsqueeze(1)
                    next_states_t = torch.FloatTensor(next_states)
                    dones_t = torch.FloatTensor(dones).unsqueeze(1)

                    #get current q values
                    current_q = policy_net(states_t).gather(1, actions_t)
                    
                    # target values
                    with torch.no_grad():
                        next_max = target_net(next_states_t).max(1)[0].unsqueeze(1)
                        td_target = rewards_t + gamma * next_max * (1 - dones_t)
                    
                    # loss calc
                    loss = nn.MSELoss()(current_q, td_target)
                    
                    # backprop and optimize + gradient clipping
                    optimizer.zero_grad()
                    loss.backward()
                    torch.nn.utils.clip_grad_norm_(policy_net.parameters(), 1.0)
                    optimizer.step()

            # update target network periodically (every n actions)
            if count % update_target_every == 0:
                target_net.load_state_dict(policy_net.state_dict())
            
            episode_reward += reward
            state = next_state

        # adding pole length
        visited_pole_lengths.append(pole_length)

        #store episode reward
        episode_returns.append(episode_reward)


        
        # update acl for rewards, performance metrics, scores, and probs
        # difficulty scores and probabilities are global updates, can be seen in the respective update methods
        acl.update_rewards(pole_length, episode_reward)
        if episode >= uniform_episode_training_cap:
            acl.update_performances(pole_length)
            acl.update_difficulties()
            acl.update_distribution()

        #only for seeing the progress
        if episode % 100 == 0:
            avg_reward = np.mean(episode_returns[-100:]) if len(episode_returns) >= 100 else np.mean(episode_returns)
            print(f"Episode {episode}/{n_episodes} | "
                  f"Avg Reward: {avg_reward:.1f} | "
                  f"Epsilon: {epsi:.3f} | "
                  )            
    
    env.close()
    
    return (policy_net, target_net, episode_returns, replay_buffer.sampled_pole_lengths, visited_pole_lengths, (acl.rewards, acl.difficulty_scores, acl.distribution))

In [ ]:
#Test the deep q learning function
# --- IGNORE ---
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)

# --- training test ---
policy_net, target_net, returns, sampled_pole_lengths, visited_pole_lengths, acl_data = deep_q_learning(
    epsilon= 0.1, 
    gamma=0.99, 
    alpha=1e-3, 
    q_network=None, 
    n_episodes=5000, 
    uniform_episode_training_cap=3000,
    pole_lengths=None, 
    env_name='CartPole-v1',
    batch_size=64, 
    buffer_capacity=50000, 
    update_target_every=1000, 
    update_weights_every=4, 
    epsilon_decay=1, 
    epsilon_min=0.05
)

print("----finished training----")
print("Last 3 episode returns:", returns[-3:] if len(returns) >= 3 else returns)

`saving training data and model`

In [ ]:
SAVE_FOLDER = "trained_nn_version_2"
MODEL_NAME = "per_strategy.pth" # .pth or .pt are common extensions
SAVE_PATH = os.path.join(SAVE_FOLDER, MODEL_NAME)
if not os.path.exists(SAVE_FOLDER):
    os.makedirs(SAVE_FOLDER)
    print(f"Created directory: {SAVE_FOLDER}")

torch.save(policy_net.state_dict(), SAVE_PATH)

_SAVE_FOLDER = "trained_nn_version_2"
_DATA_NAME = "acl_strategy_rewards.json" 
__DATA_NAME = "acl_strategy_rewards_sampled_pole_lengths.json"# .pth or .pt are common extensions
acl_rewards, acl_difficulty_scores, acl_distribution = acl_data
# ACL_REWARDS = "acl_strategy_rewards.json"
# ACL_SCORES = "acl_strategy_scores.json"
# ACL_PROBABILITY = "acl_strategy_distribution.json"
ACL_DATA = "acl_strategy_data_tuple.json"

os.makedirs(_SAVE_FOLDER, exist_ok=True)
full_path = os.path.join(_SAVE_FOLDER, _DATA_NAME)
_full_path = os.path.join(_SAVE_FOLDER, __DATA_NAME)
acl_rewards_path = os.path.join(_SAVE_FOLDER, ACL_DATA)
try:
    with open(full_path, 'w') as file:
        json.dump(returns, file, indent=4) # indent=4 makes the file nicely formatted
    print(f"List successfully saved to {full_path}")
except Exception as e:
    print(f"An error occurred: {e}")

try:
    with open(_full_path, 'w') as file:
        json.dump(sampled_pole_lengths, file, indent=4) # indent=4 makes the file nicely formatted
    print(f"List successfully saved to {_full_path}")
except Exception as e:
    print(f"An error occurred: {e}")

try:
    with open(acl_rewards_path, 'w') as file:
        json.dump(acl_data, file, indent=4) # indent=4 makes the file nicely formatted
    print(f"List successfully saved to {acl_rewards_path}")
except Exception as e:
    print(f"An error occurred: {e}")

    

NameError: name 'acl_data' is not defined

# Testing with the test script

In [19]:
from test_script import test_script

In [20]:
test_script("trained_nn_version_2/per_strategy.pth")

TypeError: test_script() takes 0 positional arguments but 1 was given